<a href="https://colab.research.google.com/github/noor-osama/headPoseEstimation-/blob/main/headPoseEstimation_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!wget http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
!unzip AFLW2000-3D.zip

--2022-06-03 16:30:50--  http://www.cbsr.ia.ac.cn/users/xiangyuzhu/projects/3DDFA/Database/AFLW2000-3D.zip
Resolving www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)... 159.226.21.60
Connecting to www.cbsr.ia.ac.cn (www.cbsr.ia.ac.cn)|159.226.21.60|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87413900 (83M) [application/x-zip-compressed]
Saving to: ‘AFLW2000-3D.zip.2’

AFLW2000-3D.zip.2   100%[===================>]  83.36M   315KB/s    in 4m 26s  

2022-06-03 16:35:17 (321 KB/s) - ‘AFLW2000-3D.zip.2’ saved [87413900/87413900]

Archive:  AFLW2000-3D.zip
replace AFLW2000/Code/DrawSolidHead.m? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
import numpy as np
import os
import scipy.io as sio
import cv2
import math
from math import cos, sin
import dlib
import glob
from pathlib import Path
import pandas as pd
from PIL import Image, ImageFilter

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error ,r2_score


In [22]:
mylist = [Path(f).stem for f in glob.glob("AFLW2000/*.mat")]
sio.loadmat(os.path.join('AFLW2000/',mylist[0]))

{'Color_Para': array([[ 0.8440023 ,  1.0719812 ,  1.2737658 , -0.11010647, -0.07766   ,
         -0.04330396,  1.        ]], dtype=float32),
 'Exp_Para': array([[-3.95290757],
        [-1.77342776],
        [-0.94872395],
        [ 0.87078476],
        [ 0.17700903],
        [ 0.0410185 ],
        [-0.79737847],
        [ 1.71817927],
        [ 0.16897713],
        [ 0.17156599],
        [ 0.66140333],
        [ 0.9052491 ],
        [ 0.32678599],
        [-1.20348187],
        [ 0.08650002],
        [-0.30113656],
        [ 0.02610228],
        [ 0.17025053],
        [-0.14090874],
        [ 0.37761379],
        [ 0.12873234],
        [-0.17973414],
        [ 0.06987645],
        [ 0.03926005],
        [ 0.57986301],
        [-0.25389097],
        [-0.0368891 ],
        [ 0.23785595],
        [-0.07355066]]),
 'Illum_Para': array([[3.22478592e-01, 2.85804391e-01, 2.30997548e-01, 1.05171955e+00,
         1.11781406e+00, 1.18432403e+00, 5.05438685e-01, 1.64901865e+00,
         2.1423595

In [23]:
def get_list_from_filenames(file_path):
    # input:    relative path to .txt file with file names
    # output:   list of relative path names
    lines = [Path(f).stem for f in glob.glob(file_path)]
    return lines

In [24]:
def get_pt3d_from_mat(mat_path):
    # Get 2D landmarks
    mat = sio.loadmat(mat_path)
    pt3d = mat['pt3d_68']
    return pt3d

In [25]:
def get_ypr_from_mat(mat_path):
    # Get yaw, pitch, roll from .mat annotation.
    # They are in radians
    mat = sio.loadmat(mat_path)
    # [pitch yaw roll tdx tdy tdz scale_factor]
    pre_pose_params = mat['Pose_Para'][0]
    # Get [pitch, yaw, roll]
    pose_params = pre_pose_params[:3]
    return pose_params

In [26]:
#prepare data

xs = ['x'+str(i) for i in range(1,69)]
ys = ['y'+str(i) for i in range(1,69)]
labels = ['pitch', 'yaw', 'roll']

column_names = xs + ys + labels
def get_data ():
  files = get_list_from_filenames("AFLW2000/*.mat")
  final = pd.DataFrame() 
  for file in files :
        d = get_pt3d_from_mat('AFLW2000/' + file)
        data = np.array(d, dtype=np.float32)
        #print(data.shape)
        df1= pd.DataFrame(data)
        df1=df1.iloc[:-1,:]
        #df1
        dfx = df1.iloc[:-1,:]
        #dfx
        dfy = pd.DataFrame(np.array(df1.iloc[-1,:]).reshape(1,-1))
        #dfy
        xy =  pd.DataFrame(pd.concat([dfx, dfy], join='inner', axis=1))
        pyr=pd.DataFrame(get_ypr_from_mat('AFLW2000/' + file).reshape(1,-1))
        #pyr
        result = pd.DataFrame(pd.concat([xy, pyr], join='inner', axis=1))
        final = final.append(result, ignore_index=True)


  final.columns = column_names
  return final 


In [18]:
#prepare dataframe 
df=get_data()
y = pd.DataFrame()
X = df.iloc[:,:-3]
pitch =pd.DataFrame(df.iloc[:,-3])
yaw =pd.DataFrame(df.iloc[:,-2])
roll =pd.DataFrame(df.iloc[:,-1])


In [30]:

#train_test_spilt
Xp_train, Xp_test, p_train, p_test = train_test_split(X, pitch, test_size=0.33, random_state=42)
# pitch prediction
p_reg =SVR(C=1.0, epsilon=0.2)
p_reg.fit(Xp_train, p_train)
p_pred = pd.DataFrame(p_reg.predict (Xp_test))
p_pred.columns = ["pitch"]
p_pred
#MSE 
mseP = mean_squared_error(p_test,p_pred)
print("MSE of pitch : %.2f" %mseP)
r2P = r2_score(p_test,p_pred)
print("R2 score of pitch : %.2f" %r2P)

##############################################################
#train_test_spilt
Xy_train, Xy_test, y_train, y_test = train_test_split(X, yaw, test_size=0.33, random_state=42)
# yaw prediction
y_reg =SVR(C=1.0, epsilon=0.2)
y_reg.fit(Xy_train, y_train)
y_pred = pd.DataFrame(y_reg.predict (Xy_test))
y_pred.columns = ["yaw"]
y_pred
#MSE 
mseY =mean_squared_error(y_test,y_pred)
print("MSE of yaw  : %.2f" %mseY)
r2Y =r2_score(y_test,y_pred)
print("R2 score of yaw : %.2f" % r2Y)

##############################################################
#train_test_spilt
Xr_train, Xr_test, r_train, r_test = train_test_split(X, roll, test_size=0.33, random_state=42)
# yaw prediction
r_reg =SVR(C=1.0, epsilon=0.2)
r_reg.fit(Xr_train, r_train)
r_pred = pd.DataFrame(r_reg.predict (Xr_test))
r_pred.columns = ["roll"]
r_pred
#MSE 
mseR =mean_squared_error(r_test,r_pred)
print("MSE of rool  : %.2f" %mseR)
r2R =r2_score(r_test,r_pred)
print("R2 score of roll : %.2f" % r2R)



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE of pitch : 0.07
R2 score of pitch : 0.65
MSE of yaw  : 0.01
R2 score of yaw : 0.98
MSE of yaw  : 0.07
R2 score of yaw : 0.67


In [31]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2

--2022-06-03 17:46:06--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M   983KB/s    in 70s     

2022-06-03 17:47:17 (889 KB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [33]:
!bzip2 -d  '/content/shape_predictor_68_face_landmarks.dat.bz2'

In [36]:
predictor=dlib.shape_predictor('/content/shape_predictor_68_face_landmarks.dat')
cnn_face_detector = dlib.get_frontal_face_detector()